# Работа с моделями Mistral

## Введение

В этом уроке вы узнаете:
- Какие бывают модели Mistral
- Для каких задач и сценариев подходит каждая модель
- Примеры кода, демонстрирующие особенности каждой модели


## Модели Mistral

В этом уроке мы рассмотрим три разные модели Mistral: **Mistral Large**, **Mistral Small** и **Mistral Nemo**.

Все эти модели доступны бесплатно на маркетплейсе моделей Github. Код в этом ноутбуке будет использовать эти модели для запуска примеров. Подробнее о том, как использовать Github Models для [прототипирования с AI-моделями](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).


## Mistral Large 2 (2407)
Mistral Large 2 — это флагманская модель от Mistral, разработанная для корпоративного использования.

Модель является улучшенной версией оригинальной Mistral Large и предлагает:
- Увеличенное окно контекста — 128k против 32k
- Лучшая производительность в задачах по математике и программированию — средняя точность 76,9% против 60,4%
- Повышенная многозадачность на разных языках — поддерживаются: английский, французский, немецкий, испанский, итальянский, португальский, нидерландский, русский, китайский, японский, корейский, арабский и хинди.

Благодаря этим возможностям, Mistral Large отлично подходит для:
- *Генерации с расширенным поиском (RAG)* — за счёт увеличенного окна контекста
- *Вызова функций* — у этой модели есть встроенная поддержка вызова функций, что позволяет интегрироваться с внешними инструментами и API. Вызовы могут выполняться как параллельно, так и последовательно, один за другим.
- *Генерации кода* — модель особенно хорошо справляется с генерацией кода на Python, Java, TypeScript и C++.


В этом примере мы используем Mistral Large 2 для реализации паттерна RAG над текстовым документом. Вопрос написан на корейском языке и касается деятельности автора до поступления в колледж.

Для создания эмбеддингов текстового документа и вопроса используется модель Cohere Embeddings. В этом примере в качестве векторного хранилища используется Python-пакет faiss.

В запросе к модели Mistral передаются как сам вопрос, так и найденные фрагменты текста, наиболее похожие на этот вопрос. Модель затем формирует ответ на естественном языке.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Small — это еще одна модель из семейства Mistral, относящаяся к категории премиум/корпоративных решений. Как следует из названия, это Маленькая Языковая Модель (SLM). Преимущества использования Mistral Small заключаются в следующем:
- Экономия средств по сравнению с языковыми моделями Mistral, такими как Mistral Large и NeMo — снижение цены на 80%
- Низкая задержка — более быстрый отклик по сравнению с LLM от Mistral
- Гибкость — может быть развернута в различных средах с меньшими ограничениями по необходимым ресурсам

Mistral Small отлично подходит для:
- Текстовых задач, таких как суммирование, анализ тональности и перевод
- Приложений, где часто поступают запросы, благодаря своей экономичности
- Задач с низкой задержкой, связанных с кодом, например, для ревью и предложений по коду


## Сравнение Mistral Small и Mistral Large

Чтобы увидеть разницу в задержке между Mistral Small и Large, выполните приведённые ниже ячейки.

Вы должны заметить разницу во времени отклика в пределах 3-5 секунд. Также обратите внимание на длину и стиль ответов на один и тот же запрос.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

В отличие от двух других моделей, рассмотренных в этом уроке, Mistral NeMo — единственная бесплатная модель с лицензией Apache2.

Её считают улучшенной версией предыдущей открытой LLM от Mistral — Mistral 7B.

Некоторые другие особенности модели NeMo:

- *Более эффективная токенизация:* Эта модель использует токенизатор Tekken вместо более распространённого tiktoken. Это обеспечивает лучшую производительность для большего количества языков и кода.

- *Доступность для дообучения:* Базовая модель доступна для дообучения. Это даёт больше гибкости в случаях, когда требуется адаптация под конкретные задачи.

- *Встроенный вызов функций* — Как и Mistral Large, эта модель обучена на вызове функций. Это делает её одной из первых открытых моделей с такой возможностью.


## Mistral NeMo

В отличие от двух других моделей, рассмотренных в этом уроке, Mistral NeMo — единственная бесплатная модель с лицензией Apache2.

Её считают улучшенной версией предыдущей открытой LLM от Mistral — Mistral 7B.

Некоторые другие особенности модели NeMo:

- *Более эффективная токенизация:* Эта модель использует токенизатор Tekken вместо более распространённого tiktoken. Это обеспечивает лучшую производительность для большего количества языков и кода.

- *Доводка (finetuning):* Базовая модель доступна для доводки. Это даёт больше гибкости для сценариев, где может потребоваться дополнительное обучение.

- *Встроенный вызов функций* — Как и Mistral Large, эта модель обучена на вызове функций. Это делает её уникальной, так как она стала одной из первых открытых моделей с такой возможностью.


### Сравнение токенизаторов

В этом примере мы рассмотрим, как Mistral NeMo обрабатывает токенизацию по сравнению с Mistral Large.

Оба примера используют один и тот же запрос, но вы должны заметить, что NeMo возвращает меньше токенов по сравнению с Mistral Large.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## На этом обучение не заканчивается — продолжайте свой путь

После завершения этого урока загляните в нашу [подборку по обучению генеративному ИИ](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst), чтобы продолжить развивать свои знания в области генеративного искусственного интеллекта!



---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные толкования, возникшие в результате использования данного перевода.
